# **Projeto: Quais fatores mais influenciam no desempenho de um aluno no ENEM e como podemos ajudá-lo a melhorar sua performance?**

**Integrantes:** Hellen Cristine Silva Rosa (RA00319076), João Victor Porto (RA00311353), Laura Gabriel Murayama (RA00319321), Maria Eduarda Bonel Iribarnegaray (RA00318891), Vinícius Ferreira de Mendonça (RA00319760), Vitória de Fátima Teixeira (RA00320578)

### **Importando bibliotecas e definindo funções importantes**

In [ ]:
SEED = 3

# loading environment variables
from dotenv import load_dotenv
load_dotenv()

# accelerating sklearn
from sklearnex import patch_sklearn
patch_sklearn()

# importing libraries
import os
import re
import psycopg2
import pandas as pd
import numpy as np
import joblib
import skops.io as sio
import matplotlib.pyplot as plt
import seaborn as sns

# importing sklearn functions
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, f1_score, precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold

In [ ]:
# establishing connection with DB
class db_connection():
    '''
    Instantiates a connection with the database.
    '''
    def __init__(self):
        self.DB_URI = os.environ.get('DB_URI')
         
    def __enter__(self):
        self.connection = psycopg2.connect(self.DB_URI)
        return self.connection
    
    def __exit__(self, *args):
        self.connection.close()

### **REQ#03-11: Treinamento de modelos**

Todos os modelos serão testados com 30% dos dados. A avaliação será feita a partir das métricas acurácia, precision, recall, F1 e matriz de confusão. Eles serão persistidos.

In [ ]:
with db_connection() as conn, conn.cursor() as cursor:
    query = '''
    SELECT *
    FROM projeto_enem.filtered_aggregated_data
    ORDER BY "NU_INSCRICAO" ASC;
    '''

    cursor.execute(query)
    complete_data_fetched = cursor.fetchall()
    complete_data_columns = tuple(desc[0] for desc in cursor.description)


complete_data = pd.DataFrame(complete_data_fetched, columns=complete_data_columns)
complete_data

In [ ]:
renda_inf_indexes = complete_data[complete_data['MICRODADOS_RENDA_PER_CAPITA'] == float('inf')].index

complete_data = complete_data.drop(index=renda_inf_indexes).reset_index(drop=True)
complete_data

#### **Pré-processamento**

**nominal:** 'MICRODADOS_TP_SEXO', 'MICRODADOS_REGIAO_ESCOLA'

**numerical:** 'NU_INSCRICAO', 'MICRODADOS_TP_FAIXA_ETARIA', 'MICRODADOS_TP_COR_RACA', 'MICRODADOS_TP_ST_CONCLUSAO', 'MICRODADOS_TP_ANO_CONCLUIU', 'MICRODADOS_TP_ESCOLA', 'MICRODADOS_TP_ENSINO', 'MICRODADOS_IN_TREINEIRO', 'MICRODADOS_TP_PRESENCA_CN', 'MICRODADOS_TP_PRESENCA_CH', 'MICRODADOS_TP_PRESENCA_LC', 'MICRODADOS_TP_PRESENCA_MT', 'MICRODADOS_NU_NOTA_CN', 'MICRODADOS_NU_NOTA_CH', 'MICRODADOS_NU_NOTA_LC', 'MICRODADOS_NU_NOTA_MT', 'MICRODADOS_NU_NOTA_REDACAO', 'MICRODADOS_TP_STATUS_REDACAO', 'MICRODADOS_NU_NOTA_COMP1', 'MICRODADOS_NU_NOTA_COMP2', 'MICRODADOS_NU_NOTA_COMP3', 'MICRODADOS_NU_NOTA_COMP4', 'MICRODADOS_NU_NOTA_COMP5', 'MICRODADOS_NU_NOTA_MEDIA', 'MICRODADOS_NU_NOTA_CONCEITO', 'MICRODADOS_NU_ACERTOS_CN', 'MICRODADOS_NU_ACERTOS_CH', 'MICRODADOS_NU_ACERTOS_LC', 'MICRODADOS_NU_ACERTOS_MT', 'MICRODADOS_NU_ACERTOS_TOTAL', 'MICRODADOS_NU_ACERTOS_MEDIO', 'MICRODADOS_TP_LINGUA', 'MICRODADOS_RENDA_PER_CAPITA', 'HABESTUDO_ST_MAT_PERC_APR', 'HABESTUDO_GES_TEMP_PLAN_EST', 'HABESTUDO_PRAT_EST_PES', 'HABESTUDO_TECN_TP_ACES', 'HABESTUDO_PROB_ROT_EST', 'HABESTUDO_DIF_INFR', 'HABESTUDO_AJUD_TERC', 'HABESTUDO_AVAL_PROP_EXP'

In [ ]:
nominal_features = list(set(complete_data.select_dtypes(include='object').columns.values))
numerical_features = list(set(complete_data.select_dtypes(exclude='object').columns.values))

In [ ]:
nominal_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder()),
])

preprocessor = ColumnTransformer(transformers=[
    ('nominal', nominal_transformer, nominal_features),
], remainder='passthrough')

In [ ]:
X = complete_data.drop(columns=['NU_INSCRICAO', 'MICRODADOS_NU_NOTA_MEDIA', 'MICRODADOS_NU_NOTA_CONCEITO'])

X_transformed = preprocessor.fit_transform(X)
y_numerical_transformed = complete_data['MICRODADOS_NU_NOTA_MEDIA']
y_categorical_transformed = complete_data['MICRODADOS_NU_NOTA_CONCEITO']

In [ ]:
preprocessor_feature_names = preprocessor.get_feature_names_out()
preprocessor_feature_names = [re.sub(r'.+__', '', item) for item in preprocessor_feature_names]

In [ ]:
X_transformed = pd.DataFrame(X_transformed, columns=preprocessor_feature_names)
X_transformed

In [ ]:
columns_to_drop = ['MICRODADOS_TP_ST_CONCLUSAO', 'MICRODADOS_TP_ANO_CONCLUIU', 'MICRODADOS_IN_TREINEIRO', 'MICRODADOS_TP_PRESENCA_CN', 'MICRODADOS_TP_PRESENCA_CH', 'MICRODADOS_TP_PRESENCA_LC', 'MICRODADOS_TP_PRESENCA_MT', 'MICRODADOS_TP_SEXO_M', 'MICRODADOS_NU_ACERTOS_MEDIO', 'MICRODADOS_NU_ACERTOS_CN', 'MICRODADOS_NU_ACERTOS_CH', 'MICRODADOS_NU_ACERTOS_LC', 'MICRODADOS_NU_ACERTOS_MT', 'MICRODADOS_NU_NOTA_CN', 'MICRODADOS_NU_NOTA_CH', 'MICRODADOS_NU_NOTA_LC', 'MICRODADOS_NU_NOTA_MT', 'MICRODADOS_NU_NOTA_REDACAO', 'MICRODADOS_TP_STATUS_REDACAO', 'MICRODADOS_NU_NOTA_COMP1', 'MICRODADOS_NU_NOTA_COMP2', 'MICRODADOS_NU_NOTA_COMP3', 'MICRODADOS_NU_NOTA_COMP4', 'MICRODADOS_NU_NOTA_COMP5', 'MICRODADOS_NU_ACERTOS_TOTAL', 'MICRODADOS_TP_LINGUA']

X_transformed = X_transformed.drop(columns=columns_to_drop).reset_index(drop=True)
X_transformed

In [ ]:
X_numerical_train, X_numerical_test, y_numerical_train, y_numerical_test = train_test_split(X_transformed, y_numerical_transformed, test_size=0.3, random_state=SEED)

X_categorical_train, X_categorical_test, y_categorical_train, y_categorical_test = train_test_split(X_transformed, y_categorical_transformed, test_size=0.3, random_state=SEED)

#### **kNN**

#### **Regressão Linear**

#### **Regressão Logística**

#### **Naive Bayes**

#### **SVM**

#### **Random Forest**

**Classificador**

In [ ]:
randomforest_classifier_pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=SEED)),
])

In [ ]:
randomforest_classifier_pipe.fit(X_categorical_train, y_categorical_train)

In [ ]:
randomforest_classifier_accuracy_scores = cross_val_score(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cv=5, scoring='balanced_accuracy')
randomforest_classifier_accuracy_scores.mean()

In [ ]:
randomforest_classifier_precision_scores = cross_val_score(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cv=5, scoring='precision_weighted')
randomforest_classifier_precision_scores.mean()

In [ ]:
randomforest_classifier_recall_scores = cross_val_score(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cv=5, scoring='recall_weighted')
randomforest_classifier_recall_scores.mean()

In [ ]:
randomforest_classifier_f1_scores = cross_val_score(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cv=5, scoring='f1_weighted')
randomforest_classifier_f1_scores.mean()

In [ ]:
ConfusionMatrixDisplay.from_estimator(randomforest_classifier_pipe, X_transformed, y_categorical_transformed, cmap='Blues')
plt.show()

In [ ]:
randomforest_classifier_feature_importances = pd.Series(randomforest_classifier_pipe['classifier'].feature_importances_, index=X_categorical_train.columns)
randomforest_classifier_feature_importances = randomforest_classifier_feature_importances.sort_values(ascending=False)
randomforest_classifier_feature_importances = randomforest_classifier_feature_importances.apply(lambda x: round(100*x, 2))
randomforest_classifier_feature_importances

**Regressor**

In [ ]:
randomforest_regressor_pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(random_state=SEED)),
])

In [ ]:
randomforest_regressor_pipe.fit(X_numerical_train, y_numerical_train)

In [ ]:
randomforest_regressor_mae_scores = -1 * cross_val_score(randomforest_regressor_pipe, X_transformed, y_numerical_transformed, cv=5, scoring='neg_mean_absolute_error')
randomforest_regressor_mae_scores.mean()

In [ ]:
randomforest_regressor_rmse_scores = -1 * cross_val_score(randomforest_regressor_pipe, X_transformed, y_numerical_transformed, cv=5, scoring='neg_root_mean_squared_error')
randomforest_regressor_rmse_scores.mean()

In [ ]:
randomforest_regressor_r2_scores = cross_val_score(randomforest_regressor_pipe, X_transformed, y_numerical_transformed, cv=5, scoring='r2')
randomforest_regressor_r2_scores.mean()

In [ ]:
randomforest_regressor_mape_scores = -1 * cross_val_score(randomforest_regressor_pipe, X_transformed, y_numerical_transformed, cv=5, scoring='neg_mean_absolute_percentage_error')
randomforest_regressor_mape_scores.mean()

In [ ]:
randomforest_regressor_feature_importances = pd.Series(randomforest_regressor_pipe['regressor'].feature_importances_, index=X_numerical_train.columns)
randomforest_regressor_feature_importances = randomforest_regressor_feature_importances.sort_values(ascending=False)
randomforest_regressor_feature_importances = randomforest_regressor_feature_importances.apply(lambda x: round(100*x, 2))
randomforest_regressor_feature_importances